In [12]:
import torch
import torch.nn as nn

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
df = pd.read_csv('../../../eICU/training/small_final.csv')

In [14]:
df.shape

(1373467, 11)

In [18]:
train_size = int(len(df) * 0.7)
test_size = len(df) - train_size
train_data, test_data = df.iloc[0:train_size], df.iloc[train_size:len(df)]
print(len(train_data), len(test_data))

961426 412041


In [19]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data.to_numpy().reshape(-1, 1))

In [20]:
print(train_data_normalized[:5])
print(train_data_normalized[-5:])

[[-0.9136081 ]
 [-0.99858412]
 [-0.9994644 ]
 [-0.9994352 ]
 [-0.99947098]]
[[-0.99948637]
 [-0.99947603]
 [-0.9994855 ]
 [-0.99948658]
 [-0.99948709]]


In [25]:
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)

In [27]:
train_window = 40

In [28]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [29]:
train_inout_seq = create_inout_sequences(train_data_normalized, train_window)

In [30]:
train_inout_seq[:5]

[(tensor([-0.9136, -0.9986, -0.9995, -0.9994, -0.9995, -0.9994, -0.9995, -0.9995,
          -0.9995, -0.9995, -0.9995, -0.9136, -0.9986, -0.9995, -0.9994, -0.9995,
          -0.9994, -0.9995, -0.9995, -0.9995, -0.9995, -0.9995, -0.9136, -0.9986,
          -0.9995, -0.9994, -0.9995, -0.9994, -0.9995, -0.9995, -0.9995, -0.9995,
          -0.9995, -0.9136, -0.9986, -0.9995, -0.9994, -0.9995, -0.9994, -0.9995]),
  tensor([-0.9995])),
 (tensor([-0.9986, -0.9995, -0.9994, -0.9995, -0.9994, -0.9995, -0.9995, -0.9995,
          -0.9995, -0.9995, -0.9136, -0.9986, -0.9995, -0.9994, -0.9995, -0.9994,
          -0.9995, -0.9995, -0.9995, -0.9995, -0.9995, -0.9136, -0.9986, -0.9995,
          -0.9994, -0.9995, -0.9994, -0.9995, -0.9995, -0.9995, -0.9995, -0.9995,
          -0.9136, -0.9986, -0.9995, -0.9994, -0.9995, -0.9994, -0.9995, -0.9995]),
  tensor([-0.9995])),
 (tensor([-0.9995, -0.9994, -0.9995, -0.9994, -0.9995, -0.9995, -0.9995, -0.9995,
          -0.9995, -0.9136, -0.9986, -0.9995, -0.9

In [31]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [32]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [33]:
print(model)

LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [34]:
epochs = 50

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%10 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

KeyboardInterrupt: 